Кирилл Лалаянц, R33352

# Лабораторная работа No6
## Критерий Найквиста и системы с запаздыванием

Вариант 6

Импорт необходимых для работы библиотек. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import control 
import sympy
import os

SAVE_PATH = 'tex-report/figs/'
os.makedirs(SAVE_PATH, exist_ok=True)

sympy.init_printing()
p = sympy.Symbol("p")
s = sympy.Symbol("s")
t = sympy.Symbol("t")
w = sympy.Symbol("w")
I = sympy.I


def get_t(end_t = 10, dt=0.001, start_t = 0):
    return np.linspace(start_t, end_t, int(end_t / dt))

## Task 1

In [ ]:
def task1_get_tf(open_roots, closed_roots):
    open_denum = closed_denum = 1
    for open_root, closed_root in zip(open_roots, closed_roots):
        open_denum *= s - open_root
        closed_denum *= s - closed_root
        
    open_num = (closed_denum - open_denum).simplify()

    W = (open_num / open_denum).simplify()
    print(f'Open loop: {sympy.latex(W.simplify())}')
    print(f'Closed loop: {sympy.latex((W / (1 + W)).simplify())}')
    tf_open = control.tf(np.array(sympy.Poly(open_num).all_coeffs(), dtype=np.float64), np.array(sympy.Poly(open_denum).all_coeffs(), dtype=np.float64))
    tf_closed = control.feedback(tf_open)
    return tf_open, tf_closed


def get_tf_and_plot(open_roots, closed_roots, save_name, fsize=50):
    tf_open, tf_closed = task1_get_tf(open_roots, closed_roots)
    count, contour = control.nyquist(tf_open, return_contour=True, plot=True)
    print(f'Nyquist count: {count}')
    plt.title(f'Loop count: {count}')
    plt.savefig(f'{SAVE_PATH}/task1_{save_name}_nyquist.jpg') 
    
    fig, ax = plt.subplots(1, 2, figsize=(20, 8))
    ax[0].scatter(open_roots.real, open_roots.imag, color = 'red', s=100)
    ax[0].grid()
    ax[0].set_title('Open loop roots', fontsize=fsize // 2)    
    ax[1].scatter(closed_roots.real, closed_roots.imag, color = 'red', s=100)
    ax[1].grid()
    ax[1].set_title('Closed loop roots', fontsize=fsize // 2)   
    ax[0].set_xlabel('Imag', fontsize=fsize // 2)
    ax[0].set_ylabel('Real', fontsize=fsize // 2)
    ax[1].set_xlabel('Imag', fontsize=fsize // 2)
    ax[1].set_ylabel('Real', fontsize=fsize // 2)
    ax[0].tick_params(axis='both', which='major', labelsize=fsize // 3)
    ax[1].tick_params(axis='both', which='major', labelsize=fsize // 3)
    ax[0].tick_params(axis='both', which='minor', labelsize=fsize // 3)
    ax[1].tick_params(axis='both', which='minor', labelsize=fsize // 3)
    plt.savefig(f'{SAVE_PATH}/task1_{save_name}_roots.jpg') 
    
    fig, ax = plt.subplots(1, 2, figsize=(20, 8))
    ts = get_t()
    ax[0].plot(ts, control.step_response(tf_open, ts).outputs, color = 'red', linewidth=6)
    ax[0].grid()
    ax[0].set_title('Open loop step', fontsize=fsize // 2)    
    ax[1].plot(ts, control.step_response(tf_closed, ts).outputs, color = 'red', linewidth=6)
    ax[1].grid()
    ax[1].set_title('Closed loop step', fontsize=fsize // 2)   
    ax[0].tick_params(axis='both', which='major', labelsize=fsize // 3)
    ax[1].tick_params(axis='both', which='major', labelsize=fsize // 3)
    ax[0].tick_params(axis='both', which='minor', labelsize=fsize // 3)
    ax[1].tick_params(axis='both', which='minor', labelsize=fsize // 3)
    ax[0].set_xlabel('t', fontsize=fsize // 2)
    ax[0].set_ylabel('y', fontsize=fsize // 2)
    ax[1].set_xlabel('t', fontsize=fsize // 2)
    ax[1].set_ylabel('y', fontsize=fsize // 2)
    plt.savefig(f'{SAVE_PATH}/task1_{save_name}_steps.jpg') 

In [ ]:
open_roots = np.array([-1, 2, 3, 4 + 1j, 4 - 1j])
closed_roots = np.array([-1, -2, 3, 4, 5])
get_tf_and_plot(open_roots, closed_roots, '1')

In [ ]:
open_roots = np.array([-1, -2, -3, -4 + 1j, -4 - 1j])
closed_roots = np.array([-1, -2, 3, 4, 5])
get_tf_and_plot(open_roots, closed_roots, '2')

In [ ]:
open_roots = np.array([-1, 2, 3, 4 + 1j, 4 - 1j])
closed_roots = np.array([-1, -2, -3, -4, -5])
get_tf_and_plot(open_roots, closed_roots, '3')

## Task2

In [ ]:
def task2(W, save_name, fsize=50):
    k_lim = control.stability_margins(W)[0]
    ks = list(np.linspace(k_lim/10, k_lim + 1, 3))
    ks.append(k_lim)
    ks.append(1)
    
    for k in ks:
        count = control.nyquist(k*W)
        plt.title(f'K = {k : .2f}, limit = {k_lim: .2f}, Nyquist count = {count}')
        plt.savefig(f'{SAVE_PATH}/task2_{save_name}_k{k: .2f}.jpg')
        plt.show()
    
    fig, ax = plt.subplots(1, 2, figsize=(20, 8))
    ts = get_t(100)
    ax[0].plot(ts, control.step_response(W * (k_lim / 2) / (1 + W * (k_lim / 2)), ts).outputs, color = 'red', linewidth=6)
    ax[0].grid()
    ax[0].set_title(f'K = 0.5k_lim = {k_lim / 2: .2f}', fontsize=fsize // 2)  
      
    ax[1].plot(ts, control.step_response(W * (k_lim * 2) / (1 + W * (k_lim * 2)), ts).outputs, color = 'red', linewidth=6)
    ax[1].grid()
    ax[1].set_title(f'K = 2k_lim = {k_lim * 2: .2f}', fontsize=fsize // 2)   
    ax[0].tick_params(axis='both', which='major', labelsize=fsize // 3)
    ax[1].tick_params(axis='both', which='major', labelsize=fsize // 3)
    ax[0].tick_params(axis='both', which='minor', labelsize=fsize // 3)
    ax[1].tick_params(axis='both', which='minor', labelsize=fsize // 3)
    ax[0].set_xlabel('t', fontsize=fsize // 2)
    ax[0].set_ylabel('y', fontsize=fsize // 2)
    ax[1].set_xlabel('t', fontsize=fsize // 2)
    ax[1].set_ylabel('y', fontsize=fsize // 2)
    plt.savefig(f'{SAVE_PATH}/task2_{save_name}_steps.jpg') 


In [ ]:
W1 = control.tf([1, -2], [1, 3, 9])
task2(W1, '1')

In [ ]:
W2 = control.tf([10, 10, 3], [10, 1, 0, 0])
task2(W2, '2')

## Task 3

In [ ]:
def task3(W, save_name, fsize=50):
    res = control.stability_margins(W)

    k_lim = np.deg2rad((res[1]) / (res[4]))
    print(k_lim, res[1], res[4])
    ks = list(np.linspace(0, k_lim + 1, 3))
    ks.append(k_lim)
    ks.append(0.5)
    
    for k in ks:
        count = control.nyquist(control.tf(*control.pade(k))*W)
        plt.title(f'delay = {k}, limit = {k_lim: .2f}, Nyquist count = {count}')
        plt.savefig(f'{SAVE_PATH}/task3_{save_name}_k{k: .2f}.jpg')
        plt.show()
    
    fig, ax = plt.subplots(1, 2, figsize=(20, 8))
    ts = get_t(100)
    W1 = control.tf(*control.pade(k_lim / 2)) * W
    ax[0].plot(ts, control.step_response(W1 / (1 + W1), ts).outputs, color = 'red', linewidth=6)
    ax[0].grid()
    ax[0].set_title(f'delay = 0.5*delay_lim = {k_lim / 2: .2f}', fontsize=fsize // 2)  
    W2 = control.tf(*control.pade(k_lim * 2)) * W
    ax[1].plot(ts, control.step_response(W2 / (1 + W2), ts).outputs, color = 'red', linewidth=6)
    ax[1].grid()
    ax[1].set_title(f'delay = 2*delay_lim = {k_lim * 2: .2f}', fontsize=fsize // 2)   
    ax[0].tick_params(axis='both', which='major', labelsize=fsize // 3)
    ax[1].tick_params(axis='both', which='major', labelsize=fsize // 3)
    ax[0].tick_params(axis='both', which='minor', labelsize=fsize // 3)
    ax[1].tick_params(axis='both', which='minor', labelsize=fsize // 3)
    ax[0].set_xlabel('t', fontsize=fsize // 2)
    ax[0].set_ylabel('y', fontsize=fsize // 2)
    ax[1].set_xlabel('t', fontsize=fsize // 2)
    ax[1].set_ylabel('y', fontsize=fsize // 2)
    plt.savefig(f'{SAVE_PATH}/task3_{save_name}_steps.jpg') 


In [ ]:
W1 = control.tf([9, 3], [1, 3, 5])
task3(W1, '1')

In [ ]:
W2 = control.tf([10, -6, 11], [10, -1, 38, 20])
task3(W2, '2')